In [2]:
setwd(paste0(Sys.getenv('R_SOURCES'), '/house_prices'))
getwd()

[1] "/mnt/hdd/develop/kaggle/R_sources/house_prices"

In [78]:
source('main.R')
house_prices$helpers$import_libs()

combined_dataset <-
    house_prices$helpers$load_data() %>%
    (house_prices$outliers$remove_outliers) %>%
    (house_prices$missing$categ$replace_with_most_common) %>%
    (house_prices$missing$categ$fix_valid) %>%
    (house_prices$missing$numeric$replace_with_zero) %>%
    mutate(
        price_log = log(SalePrice)
    ) %>%
    select(-SalePrice, -Id)

test_that("should be no NA values except SalePrice column", {
    expect_equal(sum(is.na(combined_dataset %>% select(-price_log))), 0)
})


training_dataset <- 
    combined_dataset %>% 
    filter(dataSource == 'train')

test_that("should be no NA values in training_dataset", {
    expect_equal(sum(is.na(training_dataset)), 0)
})

In [67]:
source('main.R')

house_prices$helpers$get_character_colnames(combined_dataset)

rating_for_selected <- 
    house_prices$trans$categ$calc_rating_for_selected(
        df = combined_dataset, 
        categ_vars_for_fix = house_prices$helpers$get_character_colnames(combined_dataset),
        target_var = price_log
    )

ratings_for_all <- 
    house_prices$trans$categ$calc_rating_for_all(combined_dataset, price_log)

test_that("should be equal ratings", {
    expect_equal(ratings_for_all, rating_for_selected)
})

[1] "Alley"         "BldgType"      "BsmtCond"      "BsmtExposure" 
 [5] "BsmtFinType1"  "BsmtFinType2"  "BsmtQual"      "CentralAir"   
 [9] "Condition1"    "Condition2"    "dataSource"    "Electrical"   
[13] "ExterCond"     "Exterior1st"   "Exterior2nd"   "ExterQual"    
[17] "Fence"         "FireplaceQu"   "Foundation"    "Functional"   
[21] "GarageCond"    "GarageFinish"  "GarageQual"    "GarageType"   
[25] "Heating"       "HeatingQC"     "HouseStyle"    "KitchenQual"  
[29] "LandContour"   "LandSlope"     "LotConfig"     "LotShape"     
[33] "MasVnrType"    "MiscFeature"   "MSSubClass"    "MSZoning"     
[37] "Neighborhood"  "PavedDrive"    "PoolQC"        "RoofMatl"     
[41] "RoofStyle"     "SaleCondition" "SaleType"      "Street"       
[45] "Utilities"

In [88]:
source('main.R')

ratings <- 
    house_prices$trans$categ$calc_rating_for_all(combined_dataset, price_log)

df1 <- 
    house_prices$trans$categ$rating_transform_for_selected(
        data = combined_dataset,
        columns = house_prices$helpers$get_character_colnames(combined_dataset),
        ratings = ratings
    )

df2 <-
    combined_dataset %>%
    house_prices$trans$categ$rating_transform(price_log)


test_that("should be equal", {
    expect_equal(df1, df2)
})